In [0]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.densenet import DenseNet201


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from utils import load_data, load_test_data
from utils import num_classes, epochs, batch_size

#  Grid Search Hyperparameters (DenseNet201 Fine Tuning)

In [0]:
X_train, y_train, X_valid, y_valid = load_data(Gray2RGB=True, mean_proc='DenseNet', test_size=0.1)

In [0]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


##  Tune Learning Rate

In [0]:
def create_model(learn_rate=10e-6):
    model = Sequential()

    model_name = 'DenseNet201-Fine-Tune'

    img_rows, img_cols, img_channel = 224, 224, 3
    model.add(DenseNet201(weights='imagenet', include_top=False,
                             input_shape=(img_rows, img_cols, img_channel)))

    model.add( GlobalAveragePooling2D(data_format='channels_last'))
    model.add( Dense(1024, activation='relu'))
    model.add( Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.summary()

    for layer in model.layers[:2]:
        layer.trainable = True
    for layer in model.layers[2:]:
        layer.trainable = True

    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    optimizer = keras.optimizers.Adam(lr=learn_rate)

    model_path = './saved_models/{}.h5'.format(model_name)

    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
    earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])

    batch_size = 16
    return model

In [0]:
### 這個cell可以改成所需要的Hyperparameters，請依樣畫葫蘆，記得上一個def create_model()也要改

seed = 1
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=16, verbose=0)

learn_rate= [10e-6,10e-5,10e-4]
#learn_rate= [10e-6]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1,cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))